In [1]:
import pandas as pd
import ast

In [2]:
chr = 'X'
file_path = '../predictions/split.chr' + chr + '.tsv'

data = []

with open(file_path, 'r') as file:
    for line in file:
        # Use ast.literal_eval to safely evaluate the string as a list
        entry = ast.literal_eval(line.strip())
        data.append(entry)

# Process the data
processed_data = []

for entry in data:
    position = entry[0]
    retina = entry[1]
    gtex = entry[2]
    standard = entry[3]

    number_of_predictions = len(retina)
    for i in range(number_of_predictions):
        out = [position]
        for j in retina[i].split('|'):
            out.append(j)
        for k in gtex[i].split('|')[2:]:
            out.append(k)
        for l in standard[i].split('|')[2:]:
            out.append(l)
        processed_data.append(out)


predictions = pd.DataFrame(processed_data, columns=['Position', 'Base', 'Transcript', 
                                                    'DS_AG_retina', 'DS_AL_retina', 'DS_DG_retina', 'DS_DL_retina',
                                                    'DP_AG_retina', 'DP_AL_retina', 'DP_DG_retina', 'DP_DL_retina', 
                                                    'DS_AG_gtex', 'DS_AL_gtex', 'DS_DG_gtex', 'DS_DL_gtex',
                                                    'DP_AG_gtex', 'DP_AL_gtex', 'DP_DG_gtex', 'DP_DL_gtex',
                                                    'DS_AG_standard', 'DS_AL_standard', 'DS_DG_standard', 'DS_DL_standard',
                                                    'DP_AG_standard', 'DP_AL_standard', 'DP_DG_standard', 'DP_DL_standard'])
print(len(predictions))
predictions.head()

37561


,Position,Base,Transcript,DS_AG_retina,DS_AL_retina,DS_DG_retina,DS_DL_retina,DP_AG_retina,DP_AL_retina,DP_DG_retina,...,DP_DG_gtex,DP_DL_gtex,DS_AG_standard,DS_AL_standard,DS_DG_standard,DS_DL_standard,DP_AG_standard,DP_AL_standard,DP_DG_standard,DP_DL_standard
0,9786470,G,SHROOM2,0.00,0.00,0.00,0.00,1160,2,305,...,1919,305,0.00,0.00,0.01,0.00,2695,569,305,159
1,9786613,G,SHROOM2,0.00,0.00,0.01,0.00,2552,351,162,...,162,97,0.01,0.00,0.05,0.00,2552,601,162,97
2,9786674,T,SHROOM2,0.00,0.00,0.00,0.00,-2,1637,992,...,101,36,0.00,0.00,0.01,0.00,2491,290,101,36
3,9786718,A,SHROOM2,0.00,0.00,0.00,0.00,695,2447,948,...,2661,57,0.00,0.00,0.00,0.04,496,2447,-3,57
4,9786757,G,SHROOM2,0.00,0.00,0.00,0.00,1554,207,-47,...,-47,18,0.00,0.00,0.00,0.03,207,2408,-47,18


In [3]:
for column in ['DS_AG', 'DS_AL', 'DS_DG', 'DS_DL']:
    predictions[column + '_retina'] = pd.to_numeric(predictions[column + '_retina'], errors='coerce')
    predictions[column + '_gtex'] = pd.to_numeric(predictions[column + '_gtex'], errors='coerce')
    predictions[column + '_standard'] = pd.to_numeric(predictions[column + '_standard'], errors='coerce')

threshold = 0.2

# Filter the DataFrame
only_retina = predictions[(predictions['DS_AG_retina'] > threshold) & (predictions['DS_AG_gtex'] < threshold) & (predictions['DS_AG_standard'] < threshold) |
                          (predictions['DS_AL_retina'] > threshold) & (predictions['DS_AL_gtex'] < threshold) & (predictions['DS_AL_standard'] < threshold) |
                          (predictions['DS_DG_retina'] > threshold) & (predictions['DS_DG_gtex'] < threshold) & (predictions['DS_DG_standard'] < threshold)|
                          (predictions['DS_DL_retina'] > threshold) & (predictions['DS_DL_gtex'] < threshold) & (predictions['DS_DL_standard'] < threshold)]

print(len(only_retina.drop_duplicates()))

10


In [4]:
only_retina.to_csv('../predictions/split.chr' + chr + '.filtered.tsv', sep = '\t', index = False)
only_retina

,Position,Base,Transcript,DS_AG_retina,DS_AL_retina,DS_DG_retina,DS_DL_retina,DP_AG_retina,DP_AL_retina,DP_DG_retina,...,DP_DG_gtex,DP_DL_gtex,DS_AG_standard,DS_AL_standard,DS_DG_standard,DS_DL_standard,DP_AG_standard,DP_AL_standard,DP_DG_standard,DP_DL_standard
2039,13763686,A,OFD1,0.28,0.06,0.04,0.00,30,-243,169,...,169,-776,0.15,0.04,0.03,0.00,30,-243,169,-776
2539,18660872,GC,PPEF1,0.00,0.17,0.00,0.21,1007,-3,140,...,847,140,0.00,0.00,0.00,0.00,-682,-3029,847,675
19404,32573544,A,DMD,0.00,0.18,0.00,0.25,-881,93,32,...,-54,-14,0.00,0.05,0.00,0.13,-14,-494,-54,-14
28138,38297355,G,RP5-972B16.2,0.01,0.05,0.09,0.46,1601,-71,97,...,1697,57,0.00,0.00,0.00,0.00,1643,1548,1693,57
28143,38297464,T,RPGR,0.11,0.21,0.00,0.03,-103,-12,1451,...,544,474,0.03,0.18,0.00,0.04,-103,-12,1467,-180
31757,77933090,A,ATP7A,0.24,0.00,0.25,0.00,-61,-2658,-7,...,-7,-9,0.18,0.00,0.18,0.00,-61,594,-7,-61
31758,77933090,A,PGK1,0.24,0.00,0.25,0.00,-61,-2658,-7,...,-7,-9,0.18,0.00,0.18,0.00,-61,594,-7,-61
36713,131275124,A,IGSF1,0.22,0.09,0.13,0.01,162,-87,-125,...,-29,1025,0.17,0.03,0.09,0.01,162,-87,-29,10
36973,132097340,G,FRMD7,0.27,0.14,0.17,0.00,4,-41,-74,...,-74,-280,0.16,0.17,0.23,0.00,4,-41,-74,-3081
37284,153773716,A,PLXNB3,0.17,0.00,0.22,0.01,-198,-2,-3,...,-3,-263,0.05,0.00,0.06,0.04,-198,-1556,-3,-263
